In [ ]:
import pandas as pd
import numpy as np
from eval_utils import compute_accuracy, get_summary

In [ ]:
!ls ./results/qwen2.5-32b/

In [ ]:
result_path = "./results/zero-shot/pangea/pangea-formatted.json"
full_acc = pd.read_json(compute_accuracy(result_path))

In [ ]:
summary = get_summary(full_acc, "language")
summary

In [ ]:
accuracy = np.mean(summary['accuracy'])
valid_acc = np.mean(summary['valid_acc'])
rate = 1 - sum(summary['valid_count'])/sum(summary['total_questions'])
print(f"Accuracy: {accuracy*100}")
print(f"Valid Accuracy: {valid_acc*100}")
print(f"Failure Rate: {rate*100}")
print(f"Total questions:{np.sum(summary['total_questions'])}")
print(f"Missing questions:{np.sum(summary['none_count'])}")

In [ ]:
summary[["image_type", 'valid_acc']]

In [ ]:
def print_column(summary, category):
    for value in summary[category]:
        print(np.round(value*100, decimals=1))
        
print_column(summary, "valid_acc")

In [ ]:
results = None
for model_name, p in result_paths.items():
    full_acc = pd.read_json(p)
    summary = get_summary(full_acc, "language")
    if results is None:
        results = summary[["language", "none_count"]]
        results = results.rename(columns={"none_count": model_name})
    else:
        results[model_name] = summary['none_count']

In [ ]:
#results = None
if results is None:
    results = summary[["language", "none_count"]]
    results = results.rename(columns={"none_count": "qwen2.5-vl-7b"})
else:
    results["aya"] = summary['none_count']

In [ ]:
results

In [ ]:
import json
with open("./results/closed/aya-vision/aya-formatted.json", "r") as f:
    data = json.load(f)

In [ ]:
missing = []
for sample in data:
    if sample['prediction'] not in [0,1,2,3]:
        missing.append(sample)

In [ ]:
for k in data[0].keys():
    if "prediction" in k:
        print(k)

In [ ]:
missing = []
for sample in data:
    if sample['prediction_by_pangea'] not in [0,1,2,3]:
        missing.append(sample)

# Merge Results

In [ ]:
import json
def merge_inference(path):
    data = []
    for i in range(0,4):
        json_path = path + f"results_{i}.json"
        with open(json_path, "r") as f:
            data.extend(json.load(f))
            print(json_path)
    print(f"Merged Len: {len(data)}")
    output_path = path + "qwen7b-1024t-512i-en.json"
    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Results saved in: {output_path}")

In [ ]:
merge_inference("./results/qwen2.5-7b/")

In [ ]:
def create_unique_key(sample):
    return (
        sample['language'],
        sample['country'],
        sample['file_name'],
        sample['source'],
        sample['original_question_num'],
        sample['question'],
        sample['image_png']
    )

# Create sets of unique keys for both datasets
data_keys = set(create_unique_key(sample) for sample in data)
len(data_keys)

In [ ]:
missing = []
for sample in data:
    if sample['prediction'] not in [0,1,2,3]:
        missing.append(sample)

In [ ]:
for sample in missing:
    print(sample['reasoning'])
    print("___")

In [ ]:
import json
with open("results/pangea/pangea-LLM-formatted.json", "r") as f:
    json_str = json.load(f)

In [ ]:
import re
import ast
values = []
for sample in json_str:
    for key, v in sample.items():
        match = re.search(r'\{\s*"choice":\s*.*?\s*\}', v)
        if match:
            try:
                json_choice = ast.literal_eval(match.group())
                choice = json_choice.get("choice", "").strip().upper()
                values.append({key:choice})
            except:
                print(v)

In [ ]:
with open("results/pangea/pangea-formatted_v1.json", "r") as f:
    data = json.load(f)

In [ ]:
for sample in values:
    for k,v in sample.items():
        if len(v) == 1:
            if data[int(k)]['prediction'] is None:
                data[int(k)]['prediction'] = ord(v) - ord("A")

In [ ]:
with open("./results/pangea/pangea-formatted_v1.json", "w") as f:
    json.dump(data, f, indent=2)

In [ ]:
def read_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

def write_json(file_path):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"File saved in :{file_path}")

In [ ]:
for t, sample in enumerate(data):
    if not check_if_parse(sample['reasoning']):
        print(f"Index {t}: {sample['reasoning']}")
        user_input = input("Enter a letter (A, B, C, D) or 'None' to continue: ").strip().upper()
        if user_input == 'NONE':
            continue
        elif user_input in ['A', 'B', 'C', 'D']:
            data[t]['reasoning'] = f'{{"choice": "{user_input}"}}'
        else:
            print("Invalid input. Continuing to the next item.")

In [ ]:
languages_order = [
    'English', 
    'French', 
    'German', 
    'Dutch; Flemish', 
    'Portuguese', 
    'Spanish', 
    'Arabic', 
    'Bengali', 
    'Hindi', 
    'Telugu', 
    'Nepali', 
    'Persian', 
    'Russian', 
    'Ukrainian', 
    'Croatian', 
    'Serbian', 
    'Hungarian', 
    'Lithuanian'
]

# Convert the 'language' column to a categorical type with the specified order
summary['language'] = pd.Categorical(summary['language'], categories=languages_order, ordered=True)

# Sort the DataFrame by the 'language' column
summary = summary.sort_values('language')